In [ ]:
import os
import cv2
import zipfile
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = '/home/ajith/Documents/git_repos/RBE549_CV_Projects/ajayamoorthy_p1/Phase2/Data/Train/'
if not os.path.exists(path):
    # Create a new directory because it does not exist 
    os.makedirs(path)
    print("The new directory is created!")

In [ ]:
#train_img = []
for i in range(1,5001):
    with zipfile.ZipFile('/home/ajith/Documents/git_repos/RBE549_CV_Projects/ajayamoorthy_p1/Phase2/Data/Train.zip', 'r') as zfile:
        data = zfile.read('Train/'+str(i)+'.jpg')
    img = cv2.imdecode(np.frombuffer(data, np.uint8), 1)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path+str(i)+'.jpg', img_gray)
    # train_img.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))    

In [ ]:
# plt.axis('off')
# plt.imshow(train_img[14],cmap='gray')
#plt.imshow(cv2.cvtColor(train_img[2], cv2.COLOR_BGR2GRAY))

In [ ]:
# Hp = 100
# Wp = 100
# def patch(img,Hp,Wp):
#     Patch_top_corner = int(img.shape[0]/4)
#     Patch_left_corner = int(img.shape[1]/4)
#     Patch = img[Patch_top_corner:Patch_top_corner+Hp,Patch_left_corner:Patch_left_corner+Wp,:]
#     return Patch

# Patch = patch(img,Hp,Wp)
# plt.axis('off')
# plt.imshow(cv2.cvtColor(Patch, cv2.COLOR_BGR2RGB))

In [ ]:
def patch_generator(img,PatchSize,rho):
    x_min = rho                             # min left-top x-coordinate for patch considering the -rho pertubation  
    x_max = img.shape[1]-PatchSize-rho      # max left-top x-coordinate for patch considering the patch size as well as +rho pertubation
    y_min = rho                             # min left-top y-coordinate for patch considering the -rho pertubation 
    y_max = img.shape[0]-PatchSize-rho      # max left-top y-coordinate for patch considering the patch size as well as +rho pertubation

    x_patchA = np.random.randint(x_min,x_max) # Considering a random left-top x-coordinate for Patch-A within limits - (x_min,x_max)
    y_patchA = np.random.randint(y_min,y_max) # Considering a random left-top y-coordinate for Patch-A within limits - (y_min,y_max)

    # Clockwise corner co-ordinates of Patch A
    PatchA_corners = [[x_patchA             , y_patchA],            # Left-Top
                      [x_patchA+PatchSize   , y_patchA],            # Right-Top
                      [x_patchA+PatchSize   , y_patchA+PatchSize],  # Right-Bottom
                      [x_patchA             , y_patchA+PatchSize]]  # Left-Bottom
                      
    PatchB_corners = []
    for c in PatchA_corners:
        PatchB_corners.append([c[0] + np.random.randint(-rho,rho), c[1] + np.random.randint(-rho,rho)])

    PatchA_corners = np.float32(PatchA_corners)
    PatchB_corners = np.float32(PatchB_corners)
    
    # Calgulating the Transformation Matrix from A to B and then from B to A
    H_AB = cv2.getPerspectiveTransform(PatchA_corners,PatchB_corners)
    H_BA = np.linalg.inv(H_AB)
    img_warped = cv2.warpPerspective(img, H_BA, (img.shape[1],img.shape[0]))
    # print(img.shape)
    # print(img_warped.shape)

    # print(PatchA_corners)
    PatchA  =        img[y_patchA:(y_patchA+PatchSize),x_patchA:(x_patchA+PatchSize)]
    PatchB  = img_warped[y_patchA:(y_patchA+PatchSize),x_patchA:(x_patchA+PatchSize)]
    H4pt    = PatchB_corners - PatchA_corners
    return PatchA, PatchB, H4pt

# data_path = '/home/ajith/Documents/git_repos/RBE549_CV_Projects/ajayamoorthy_p1/Phase2/Data/Training_Data/'
# if not os.path.exists(data_path):
#     # Create a new directory because it does not exist 
#     os.makedirs(data_path)
#     print("The new directory is created!")

for i in range(1,5001):
    rho = 16  # In pixels
    patch_size = 64 # Dimension of the patch (The final patch dimensions = patch_size x patch_size)
    PatchA, PatchB, H4Pt = patch_generator(cv2.imread(path+str(i)+'.jpg',cv2.IMREAD_GRAYSCALE),patch_size,rho)
    # print(PatchA.shape)
    # print(PatchB.shape)
    if i == 1:
        training_imgs = np.dstack((PatchA,PatchB))
        training_label = H4Pt
        # print(training_imgs.shape)
    else:
        training_imgs = np.dstack((training_imgs,PatchA))
        training_imgs = np.dstack((training_imgs,PatchB))
        training_label = np.dstack((training_label,H4Pt))
        # print(training_imgs.shape)
    # cv2.imwrite(data_path+str(i)+'a.jpg',PatchA)
    # cv2.imwrite(data_path+str(i)+'b.jpg',PatchB)

In [ ]:
training_imgs.shape

In [ ]:
# def patch_generator(img,PatchSize,rho):
#     x_min = rho                             # min left-top x-coordinate for patch considering the -rho pertubation  
#     x_max = img.shape[1]-PatchSize-rho      # max left-top x-coordinate for patch considering the patch size as well as +rho pertubation
#     y_min = rho                             # min left-top y-coordinate for patch considering the -rho pertubation 
#     y_max = img.shape[0]-PatchSize-rho      # max left-top y-coordinate for patch considering the patch size as well as +rho pertubation

#     x_patchA = np.random.randint(x_min,x_max) # Considering a random left-top x-coordinate for Patch-A within limits - (x_min,x_max)
#     y_patchA = np.random.randint(y_min,y_max) # Considering a random left-top y-coordinate for Patch-A within limits - (y_min,y_max)

#     # Clockwise corner co-ordinates of Patch A
#     PatchA_corners = [[x_patchA             , y_patchA],            # Left-Top
#                       [x_patchA+PatchSize   , y_patchA],            # Right-Top
#                       [x_patchA+PatchSize   , y_patchA+PatchSize],  # Right-Bottom
#                       [x_patchA             , y_patchA+PatchSize]]  # Left-Bottom
                      
#     PatchB_corners = []
#     for c in PatchA_corners:
#         PatchB_corners.append([c[0] + np.random.randint(-rho,rho), c[1] + np.random.randint(-rho,rho)])

#     PatchA_corners = np.float32(PatchA_corners)
#     PatchB_corners = np.float32(PatchB_corners)
    
#     # Calgulating the Transformation Matrix from A to B and then from B to A
#     H_AB = cv2.getPerspectiveTransform(PatchA_corners,PatchB_corners)
#     H_BA = np.linalg.inv(H_AB)
#     img_warped = cv2.warpPerspective(img, H_BA, (img.shape[1],img.shape[0]))

#     # print(PatchA_corners)
#     PatchA =        img[x_patchA:(x_patchA+PatchSize), y_patchA:(y_patchA+PatchSize)]
#     PatchB = img_warped[x_patchA:(x_patchA+PatchSize), y_patchA:(y_patchA+PatchSize)]
#     return PatchA_corners, PatchB_corners, PatchA, PatchB, img, img_warped, H_BA

# for i in range(1,5001):
#     rho = 16  # In pixels
#     patch_size = 64 # Dimension of the patch (The final patch dimensions = patch_size x patch_size = 128 x 128 Input to network)
#     patchA_corner, patchB_corner, PatchA, PatchB, img, img_warped, Hba = patch_generator(cv2.imread(path+str(i)+'.jpg'),patch_size,rho)
#     if i == 1:
#         Train_data = np.dstack((PatchA,PatchB))
#         Train_label = Hba
#     else:
#         Train_data = np.dstack((Train_data,PatchA))
#         Train_data = np.dstack((Train_data,PatchB))
#         Train_label = np.dstack((Train_label,Hba))
#     # print(PatchA.shape)
#     # print(PatchB.shape)
#     # print(Train_data.shape)
#     # print(Train_label.shape)
#     print(i)

# print(Train_data.shape)
# print(Train_label.shape)

# plt.axis('off')
# plt.imshow(cv2.cvtColor(PatchA,cv2.COLOR_BGR2RGB))
# plt.show()
# plt.axis('off')
# plt.imshow(cv2.cvtColor(PatchB,cv2.COLOR_BGR2RGB))
# plt.show()

# # Plotting the original
# i=0
# j=1
# while i < len(patchA_corner):
#     plt.axis('off')
#     plt.scatter(patchA_corner[i][0],patchA_corner[i][1],c ='b')
#     plt.scatter(patchB_corner[i][0],patchB_corner[i][1],c ='r')
#     plt.plot([patchA_corner[i][0],patchA_corner[j][0]],[patchA_corner[i][1],patchA_corner[j][1]],c ='b')
#     plt.plot([patchB_corner[i][0],patchB_corner[j][0]],[patchB_corner[i][1],patchB_corner[j][1]],c ='r')
#     plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
#     i=i+1
#     j=j+1
#     if j%len(patchA_corner) == 0:
#         i = len(patchA_corner)-1
#         j = 0

# plt.show()

# # Plotting the Warp
# i=0
# j=1
# while i < len(patchA_corner):
#     plt.axis('off')
#     plt.scatter(patchA_corner[i][0],patchA_corner[i][1],c ='r')
#     plt.plot([patchA_corner[i][0],patchA_corner[j][0]],[patchA_corner[i][1],patchA_corner[j][1]],c ='r')
#     plt.imshow(cv2.cvtColor(img_warped,cv2.COLOR_BGR2RGB))
#     i=i+1
#     j=j+1
#     if j%len(patchA_corner) == 0:
#         i = len(patchA_corner)-1
#         j = 0

# plt.show()